In [1]:
# %load_ext tensorboard
# Clear any logs from previous runs
# !rm -rf ./logs/

In [2]:
import numpy as np
import pandas as pd
import time, os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical

from tensorboard.plugins.hparams import api as hp

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model

import astropy
from astropy.io import fits

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split 

import random
from math import floor
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from scipy.integrate import odeint
import scipy
import pandas as pd
np.random.seed(0)

2022-12-09 16:43:59.472322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 16:43:59.563312: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 16:43:59.563330: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-09 16:44:00.145127: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [3]:
with fits.open('https://github.com/igomezv/cosmo_tools/blob/main/COB_22/Viviana_Acquaviva/DEEP2_uniq_Terapix_Subaru_v1.fits?raw=true') as data:
    df = pd.DataFrame(np.array(data[1].data).byteswap().newbyteorder()) #see https://numpy.org/devdocs/user/basics.byteswapping.html#changing-byte-ordering

In [4]:
df.head()

,objno_deep2,ra_deep2,dec_deep2,magb,magr,magi,pgal,sfd_ebv,class,subclass,...,ra_subaru,dec_subaru,y,yerr,y_apercor,yerr_aper,yerr_apercor,y(sexflag),y_radius_arcsec,subaru_source
0,11001673,213.868704,51.956445,23.487745,23.143082,22.582092,1.000000,0.010943,b'',b'',...,213.868626,51.956443,21.869627,0.060918,21.926356,0.041955,0.141778,3.0,0.656514,1
1,11001699,213.810471,51.942316,22.067692,20.034674,19.545080,3.000000,0.011014,b'GALAXY',b'',...,213.810455,51.942321,18.757229,0.005813,18.811085,0.004386,0.050987,3.0,0.744269,1
2,11001770,213.848431,51.948876,24.144438,24.103180,24.020006,3.000000,0.010856,b'GALAXY',b'',...,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.0,-99.000000,-99
3,11001800,213.831758,51.952548,25.336836,23.508480,23.081087,0.509809,0.010823,b'GALAXY',b'',...,213.831766,51.952544,22.404269,0.088970,22.535600,0.053497,0.094733,0.0,0.455820,1
4,11001860,213.832550,51.954174,24.382738,23.401484,22.572845,3.000000,0.010827,b'GALAXY',b'',...,213.832574,51.954175,22.242717,0.070760,22.100980,0.033256,0.073067,0.0,0.442022,1


In [5]:
features = df[['u_apercor', 'g_apercor', 'r_apercor', 'i_apercor', 'z_apercor','y_apercor']]
features.head()

,u_apercor,g_apercor,r_apercor,i_apercor,z_apercor,y_apercor
0,23.543491,23.430495,23.100311,22.768970,22.223810,21.926356
1,30.848978,28.989668,19.027422,99.000000,99.000000,18.811085
2,24.324670,24.273606,24.150319,23.446252,23.574236,-99.000000
3,99.000000,24.804309,23.636544,23.009222,22.689591,22.535600
4,24.362068,24.136913,23.490342,22.777181,22.319676,22.100980


In [6]:
target = df['zhelio']
target.head()

0    1.115261
1    0.290608
2    0.605744
3    1.306796
4    0.957669
Name: zhelio, dtype: float32

In [7]:
mags = df[['u_apercor', 'g_apercor', 'r_apercor', 'i_apercor', 'z_apercor','y_apercor','zquality','cfhtls_source']]

In [8]:
mags = mags[mags['zquality'] >= 3]

mags.shape

(16857, 8)

In [9]:
mags = mags[mags > -10].dropna()
mags = mags[mags < 90].dropna()

In [10]:
mags = mags[mags['cfhtls_source'] == 0]

In [11]:
params = ['u_apercor', 'g_apercor', 'r_apercor', 'i_apercor', 'z_apercor','y_apercor']
features_ext = mags.copy()
for i, name1 in enumerate(params):
    for j, name2 in enumerate(params):
        if i >=j: continue #build only one pair, avoid zero colors
        features_ext[name1 + '-' + name2] = features[name1] - features[name2]

In [12]:
features_ext.head()

,u_apercor,g_apercor,r_apercor,i_apercor,z_apercor,y_apercor,zquality,cfhtls_source,u_apercor-g_apercor,u_apercor-r_apercor,...,g_apercor-r_apercor,g_apercor-i_apercor,g_apercor-z_apercor,g_apercor-y_apercor,r_apercor-i_apercor,r_apercor-z_apercor,r_apercor-y_apercor,i_apercor-z_apercor,i_apercor-y_apercor,z_apercor-y_apercor
1251,24.244393,23.979583,23.522136,22.911041,22.525773,22.329098,4,0.0,0.264811,0.722258,...,0.457447,1.068542,1.453810,1.650485,0.611094,0.996362,1.193037,0.385268,0.581943,0.196675
1253,24.489104,23.916151,22.923651,21.873752,21.306495,21.251440,4,0.0,0.572953,1.565453,...,0.992500,2.042399,2.609656,2.664711,1.049899,1.617157,1.672212,0.567258,0.622312,0.055055
1261,24.873959,22.973893,21.465850,20.788420,20.462283,20.413696,3,0.0,1.900066,3.408108,...,1.508042,2.185473,2.511610,2.560196,0.677430,1.003568,1.052154,0.326138,0.374724,0.048586
1271,24.529042,24.338631,23.891189,23.206102,22.989344,23.112382,4,0.0,0.190411,0.637853,...,0.447442,1.132529,1.349287,1.226250,0.685087,0.901845,0.778808,0.216758,0.093721,-0.123037
1272,23.641180,23.387447,22.975301,22.235199,21.809658,21.559483,4,0.0,0.253733,0.665879,...,0.412146,1.152248,1.577789,1.827964,0.740102,1.165643,1.415818,0.425541,0.675717,0.250175


In [13]:
target = target[features_ext.index]

In [14]:
features_paper = features_ext[['u_apercor-g_apercor','g_apercor-r_apercor', \
            'r_apercor-i_apercor','i_apercor-z_apercor','z_apercor-y_apercor','i_apercor']]

In [15]:
X_train, X_val, Y_train, Y_val = train_test_split(features_paper, target, test_size=0.2)

In [16]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                   min_delta=0.0,
                                   patience=200,
                                   restore_best_weights=True, verbose=False)
                                   ]

n_cols = 1
epochs=200

In [22]:
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([8, 16, 32, 64]))
HP_LAYERS =    hp.HParam('layers', hp.Discrete([3, 4]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([100, 200]))
HP_LEARNING  = hp.HParam('learning_rate', hp.Discrete([1e-4, 1e-3]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))

In [23]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning3').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_LAYERS,
                 HP_NUM_UNITS,
                 HP_LEARNING, 
                 HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

In [24]:
def train_test_model(hparams):    
    
    # Train LSTM model and predict on validation set
    model = tf.keras.Sequential()
    model.add(Dense(int(X_train.shape[1])))
#     model.add(Dense(hparams[HP_NUM_UNITS], input_shape=(int(X_train.shape[1]),)))
    
    for i in range(hparams[HP_LAYERS]):        
        model.add(Dense(hparams[HP_NUM_UNITS], activation='relu'))
    model.add(Dense(2, activation='linear'))
     
    optimizer = tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING], beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(
            optimizer=optimizer,
            loss='mse', 
            metrics=['mean_squared_error'])
    
    # Run with 1 epoch to speed things up for demo purposes

    model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_val, Y_val),
              callbacks=callbacks, batch_size=hparams[HP_BATCHSIZE], shuffle=False, verbose=0)

    _, loss = model.evaluate(X_val, Y_val)
    
    return loss

In [25]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)
        return loss

In [26]:
session_num = 0
datos = []

for deep_layers in HP_LAYERS.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for learning_rate in HP_LEARNING.domain.values:
            for batch_size in HP_BATCHSIZE.domain.values:
                t = time.time()
                hparams = {

                    HP_LAYERS: deep_layers,
                    HP_NUM_UNITS: num_units,
                    HP_LEARNING: learning_rate,
                    HP_BATCHSIZE: batch_size,
                }
                run_name = "run-%d" % session_num
                print('\n--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                score = run('logs/hparam_tuning3/' + run_name, hparams)
                t = time.time()-t
                session_num += 1
                print("Loss:", score, "Tiempo transcurrido:", t)
            
            datos.append([deep_layers, num_units, learning_rate, batch_size, score, t])

print(session_num)


--- Starting trial: run-0
{'layers': 3, 'num_units': 100, 'learning_rate': 0.0001, 'batch_size': 8}
40/40 [==============================] - 0s 955us/step - loss: 0.0399 - mean_squared_error: 0.0399
Loss: 0.039879266172647476 Tiempo transcurrido: 137.21259999275208

--- Starting trial: run-1
{'layers': 3, 'num_units': 100, 'learning_rate': 0.0001, 'batch_size': 16}
40/40 [==============================] - 0s 842us/step - loss: 0.0391 - mean_squared_error: 0.0391
Loss: 0.03907215595245361 Tiempo transcurrido: 75.11359119415283

--- Starting trial: run-2
{'layers': 3, 'num_units': 100, 'learning_rate': 0.0001, 'batch_size': 32}
40/40 [==============================] - 0s 813us/step - loss: 0.0322 - mean_squared_error: 0.0322
Loss: 0.03218049928545952 Tiempo transcurrido: 40.83356690406799

--- Starting trial: run-3
{'layers': 3, 'num_units': 100, 'learning_rate': 0.0001, 'batch_size': 64}
40/40 [==============================] - 0s 836us/step - loss: 0.0396 - mean_squared_error: 0.0396


Loss: 0.02720065601170063 Tiempo transcurrido: 46.750805616378784

--- Starting trial: run-31
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 64}
40/40 [==============================] - 0s 747us/step - loss: 0.0229 - mean_squared_error: 0.0229
Loss: 0.022859521210193634 Tiempo transcurrido: 32.78250789642334
32


In [27]:
filename = "historial_photoz_tunning.txt"
df = pd.DataFrame(datos, columns = ["Deep size", "Num units", "Learning rate", "Batch size", "MSE", "Tiempo de ejecución"])

df.sort_values(by=["MSE", "Tiempo de ejecución"], ascending=[True, True], ignore_index=True, inplace=True)

df.to_csv(filename, header=True, index=False, sep='\t', mode='w') # a=append, w=overwrite

In [28]:
df.head(2)

,Deep size,Num units,Learning rate,Batch size,MSE,Tiempo de ejecución
0,4,200,0.001,64,0.022860,32.782508
1,4,100,0.001,64,0.023669,28.649291


In [29]:
total_time = float(np.sum(df[["Tiempo de ejecución"]])/60)

print("Elapsed time {:.3f} minutes".format(total_time))

Elapsed time 4.018 minutes


In [30]:
df["Tiempo de ejecución"]

0    32.782508
1    28.649291
2    25.105375
3    31.389440
4    32.980246
5    27.168419
6    24.484121
7    38.493699
Name: Tiempo de ejecución, dtype: float64